# Task 4

When performing clustering, categorical features become problematic, because the usual distance for categorical features gives bad results using, for instance K-Means.

That distance is given by
$$d(a, b) = \left\{
\begin{array}{rcl}
     1 & \mbox{ , if } & a = b
  \\ 0 & \mbox{ , if } & a\neq b
\end{array}
\right.$$
In order to adapt K-Means to the case of categorical features, there is an algorithm called K-Modes that is able to handle such features in an efficient way.

The documentation of kmodes package is available here https://pypi.org/project/kmodes/. In order to install it, you will have to use pip because it is not available in any Anaconda package repository. In general, it is not recommended, for Anaconda users, to install packages using pip. In this case, I had never have problems with this package.


The dataset 'USCrimeMDLP.csv' contains 300 samples and 79 categorical features, plus a class feature. Use that dataset for the following exercises:
1.	Split the data into train and test, keeping 200 samples for training. Use stratification. Always use random_state=0.
<br><br>
2.	Use K-Means algorithm with 2clusters, and evaluate it using the area under the ROC curve (AUC) as external measure (we can do it because the class column is available). Take into account that the classes are {-1, 1} and the clusters names are {0, 1}. Did you get a strange value? Why can AUC be so low? Correct the problem. (Hint: the prediction made by clustering assigns the name of the cluster as the class, but the names are just tag names). Is the K-Means algorithm adequate in this case?
<br><br>
3.	Consider the clustering you have obtained as a classification algorithm, i.e. each cluster predicting a class. Apart from the AUC, obtained above, calculate the classification report, area under the ROC curve (AUC), and confusion matrix. 
<br><br>
4.	Compare these results with the classification performed with random forest (n_estimators=100), and with SVC (C=2.0) and NuSVC (nu=0.001).
<br><br>
5.	Use K-Modes algorithm with 2 clusters, evaluating it in the same way as with K-Means. Comparing with the result in exercise 2, should we discard kmodes as an unsupervised classification procedure?
<br><br>
6.	Make summary comments of all the exercises as a general conclusion.

## Solution

### 1. Split the data into train and test, keeping 200 samples for training. Use stratification. Always use random_state=0. 

In [1]:
import pandas as pd

df = pd.read_csv('USCrimeMDLP.csv')
df.head()

,v1,v3,v4,v6,v8,v9,v11,v12,v13,v14,...,v89,v90,v91,v92,v94,v97,v98,v99,v100,class
0,0,1,3,0,1,0,0,1,1,0,...,0,0,1,0,0,0,0,1,1,-1
1,0,0,1,0,1,1,1,1,0,0,...,0,1,1,0,0,0,0,1,0,1
2,0,2,0,0,1,0,1,0,0,0,...,0,1,1,0,0,0,0,0,0,-1
3,0,2,0,0,1,1,1,1,1,2,...,1,1,1,0,1,0,0,1,0,-1
4,0,1,3,0,0,0,1,0,1,0,...,0,1,1,0,0,0,0,0,0,-1


In [2]:
df_train = df[:200]
df_test = df[-100:]
print('Train shape: ' + str(df_train.shape))
print('Test shape: ' + str(df_test.shape))

Train shape: (200, 80)
Test shape: (100, 80)


In [3]:
X_train = df_train.iloc[:,:-1]
y_train = df_train.iloc[:,-1]

X_test = df_test.iloc[:,:-1]
y_test = df_test.iloc[:,-1]

### 2. Use K-Means algorithm with 2clusters, and evaluate it using the area under the ROC curve (AUC) as external measure (we can do it because the class column is available). Take into account that the classes are {-1, 1} and the clusters names are {0, 1}. Did you get a strange value? Why can AUC be so low? Correct the problem. (Hint: the prediction made by clustering assigns the name of the cluster as the class, but the names are just tag names). Is the K-Means algorithm adequate in this case?

In [9]:
from kmodes.kmodes import KModes

km = KModes(n_clusters=2)
clusters = km.fit(X_train)

km.cluster_centroids_

array([[0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        1, 1, 0, 0, 0, 1, 0, 2, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1,
        0, 0, 0, 0, 1, 0, 2, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 1, 3, 0, 0, 0, 1, 1, 1, 0, 0, 2, 2, 0, 2, 1, 2, 1, 2, 1, 2, 1,
        0, 0, 0, 1, 0, 1, 1, 1, 0, 2, 1, 3, 3, 3, 3, 2, 2, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 1, 0, 0, 3, 0, 1, 1, 2, 1, 3, 2, 0, 0, 1, 1, 0,
        0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]], dtype=int64)

### 3. Consider the clustering you have obtained as a classification algorithm, i.e. each cluster predicting a class. Apart from the AUC, obtained above, calculate the classification report, area under the ROC curve (AUC), and confusion matrix.

In [ ]:
# 3.

### 4. Compare these results with the classification performed with random forest (n_estimators=100), and with SVC (C=2.0) and NuSVC (nu=0.001). 

In [ ]:
# 4.

### 5. Use K-Modes algorithm with 2 clusters, evaluating it in the same way as with K-Means. Comparing with the result in exercise 2, should we discard kmodes as an unsupervised classification procedure?

In [ ]:
# 5.

### 6. Make summary comments of all the exercises as a general conclusion.

In [ ]:
# 6.